# PDF text Extraction

ML6 use case. 20 pdf documents with product Safety hazard information.


Safety Data Sheet (SDS) information:

https://en.wikipedia.org/wiki/Safety_data_sheet

https://ekos.gda.pl/en/article/baza-wiedzy/safety-data-sheet-annex-ii.html

http://chemicalsafety.com/sds-search/

https://www.msdsonline.com/sds-search/

https://comply1.com/compliance/ghs/MSDS-SDS-Comparison

https://www.everysafeuse.com/chemparser/     - Commercial product that converts SDS into json - json output example


https://echa.europa.eu/-/guidance-on-the-compilation-of-safety-data-sheets


### test file

In [1]:
InputFile = 'data/23114.pdf'

### Extract PDF file as one single text string

In [9]:
import PyPDF2 
  
import PyPDF2 
   
pdfFileObj = open(InputFile, 'rb') 
  
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
  
print(f"Number of pages in file '{InputFile}' = {pdfReader.numPages}\n") 

pageObj = pdfReader.getPage(0) 
  
PdfText = pageObj.extractText() 

pdfFileObj.close()

print(PdfText)

Number of pages in file 'data/23114.pdf' = 10

   Material Safety Data Sheet 
  LUPEROX® P
      Product code: 051000
   Version 2.0
 Issued on: 07/11/2011
 Page: 1 / 10
     1. PRODUCT AND COMPANY IDENTIFICATION
  Company
  
Arkema Inc.
 900 First Avenue 
King of Prussia, Pennsylvania 19406 

 Functional Additives
  
Customer Service Telephone Number: 
(800) 331-7654 
(Monday through Friday, 8:30 AM to 5:30 PM EST) 
 Emergency Information
  
Transportation:
 CHEMTREC: (800) 424-9300 
(24 hrs., 7 days a week) 
Medical: 
Rocky Mountain Poison Center:
 (866) 767-5089 
(24 hrs., 7 days a week) 
 
Product Information
  
Product name:
 LUPEROX® P
 Synonyms: 
Peroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate 
Molecular formula:
 C11 H14 O3
 Chemical family:
 Organic peroxide - peroxyesters
 Product use: 
Initiator
  2. HAZARDS IDENTIFICATION
  Emergency Overview
  Color:
 Colourless to yellow.
 Physical state: 
liquid 
 Odor: unpleasant 
 CAUTION! 
ORGANIC PEROXIDE. 
HAZARDOUS DECO

### Extract SDS Section

In [25]:
Header1 = "1. PRODUCT AND COMPANY IDENTIFICATION"
Header2 = "2. HAZARDS IDENTIFICATION"

In [27]:
print(PdfText.count(Header1))
PdfText.count(Header2)

1


1

In [28]:
# Locate begin and end of section header

import re

count = 1
for m in re.finditer(Header1, PdfText):
         print(f'Occurence {count} : {m.start()}, {m.end()}\n')
         count+=1

Occurence 1 : 128, 165



In [29]:
def FindOccurences(Text, SubString):
    # Find all Occurences of SubString in text. Return dict with begin and end position in Text for each occurence
    
    count = 1
    Dict = {}
    for m in re.finditer(SubString, Text):
         Dict[count] = (m.start(), m.end())
            
    return(Dict)

In [30]:
def VerifyOccurence(AllHeaders):
    # Check if SubString occurs in Text.
    
    if len(AllHeaders) == 0:
        s = "No Occurences"
    elif len(AllHeaders) > 1:
        s = "Multiple Occurences"
    else:
        s = "OK - Single Occurence"
    
    return(s)   

In [33]:
AllHeaders1 = FindOccurences(PdfText, Header1)

print(VerifyOccurence(AllHeaders1))
    
print(AllHeaders1)

OK - Single Occurence
{1: (128, 165)}


In [48]:
def ExtractText(Text, Begin, End):
    # Extract text Segment between 2 position indexes in a text string
    
    return(Text[Begin:End])

In [16]:
ExtractText(PdfText, 128, 165)

'1. PRODUCT AND COMPANY IDENTIFICATION'

In [40]:
# Locate begin and end of section "1. PRODUCT AND COMPANY IDENTIFICATION"
#
# Starts with the first postition after this header, ends with the last position before the next header

AllHeaders1 = FindOccurences(PdfText, Header1)
AllHeaders2 = FindOccurences(PdfText, Header2)

print(VerifyOccurence(AllHeaders1))
print(VerifyOccurence(AllHeaders2))

print(AllHeaders1[1][1], AllHeaders2[1][0])

OK - Single Occurence
OK - Single Occurence
165 785


In [41]:
Section1 = ExtractText(PdfText, AllHeaders1[1][1] + 1, AllHeaders2[1][0] - 1)

print(Section1)

  Company
  
Arkema Inc.
 900 First Avenue 
King of Prussia, Pennsylvania 19406 

 Functional Additives
  
Customer Service Telephone Number: 
(800) 331-7654 
(Monday through Friday, 8:30 AM to 5:30 PM EST) 
 Emergency Information
  
Transportation:
 CHEMTREC: (800) 424-9300 
(24 hrs., 7 days a week) 
Medical: 
Rocky Mountain Poison Center:
 (866) 767-5089 
(24 hrs., 7 days a week) 
 
Product Information
  
Product name:
 LUPEROX® P
 Synonyms: 
Peroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate 
Molecular formula:
 C11 H14 O3
 Chemical family:
 Organic peroxide - peroxyesters
 Product use: 
Initiator
 


### Section 0

The initial text that comes before section 1

In [49]:
# Extract Section before Header "1. PRODUCT AND COMPANY IDENTIFICATION"

Section0 = ExtractText(PdfText, 0, AllHeaders1[1][0] - 1)

print(Section0)

   Material Safety Data Sheet 
  LUPEROX® P
      Product code: 051000
   Version 2.0
 Issued on: 07/11/2011
 Page: 1 / 10
    


### Extract Entire PDF file


In [50]:
def extract_raw_text(file):

    pdfFileObj = open(file, "rb") 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

    raw_text = ""
    for n in range(pdfReader.numPages):
        raw_text += pdfReader.getPage(n).extractText()

    ##raw_text = raw_text.split("\n") 

    pdfFileObj.close() 
    return raw_text

In [52]:
PDFfile = extract_raw_text(InputFile)
print(type(PDFfile))

<class 'str'>


In [53]:
print(PDFfile.count(Header1))
PDFfile.count(Header2)

1


1

In [54]:
# Locate begin and end of section "1. PRODUCT AND COMPANY IDENTIFICATION"
#
# Starts with the first postition after this header, ends with the last position before the next header

AllHeaders1 = FindOccurences(PDFfile, Header1)
AllHeaders2 = FindOccurences(PDFfile, Header2)

print(VerifyOccurence(AllHeaders1))
print(VerifyOccurence(AllHeaders2))

print(AllHeaders1[1][1], AllHeaders2[1][0])

OK - Single Occurence
OK - Single Occurence
165 785


In [56]:
Section1 = ExtractText(PDFfile, AllHeaders1[1][1] + 1, AllHeaders2[1][0] - 1)

print(Section1)

  Company
  
Arkema Inc.
 900 First Avenue 
King of Prussia, Pennsylvania 19406 

 Functional Additives
  
Customer Service Telephone Number: 
(800) 331-7654 
(Monday through Friday, 8:30 AM to 5:30 PM EST) 
 Emergency Information
  
Transportation:
 CHEMTREC: (800) 424-9300 
(24 hrs., 7 days a week) 
Medical: 
Rocky Mountain Poison Center:
 (866) 767-5089 
(24 hrs., 7 days a week) 
 
Product Information
  
Product name:
 LUPEROX® P
 Synonyms: 
Peroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate 
Molecular formula:
 C11 H14 O3
 Chemical family:
 Organic peroxide - peroxyesters
 Product use: 
Initiator
 


In [57]:
# Extract Section before Header "1. PRODUCT AND COMPANY IDENTIFICATION"

Section0 = ExtractText(PDFfile, 0, AllHeaders1[1][0] - 1)

print(Section0)

   Material Safety Data Sheet 
  LUPEROX® P
      Product code: 051000
   Version 2.0
 Issued on: 07/11/2011
 Page: 1 / 10
    
